In [1]:
import json
import pandas as pd
import ast
import datetime
from glob import glob

In [7]:
files = sorted(glob("merged_*.csv"))

def get_date(s):
    return s.split("merged_")[1].split(".csv")[0]

infile1 = files[0]
infile2 = files[1]
col_to_export = "venezuelans"

df1 = pd.read_csv(infile1)
df2 = pd.read_csv(infile2)
merged = pd.merge(df1[["Location","LocationHierarchy","Frequency", col_to_export]], df2[["Location","LocationHierarchy","Frequency", col_to_export]], 
                  on=["Location","LocationHierarchy","Frequency"], suffixes=(get_date(files[0]), (get_date(files[1]))), how="outer")

for i in range(2, len(files)):
    tmp = pd.read_csv(files[i])
    merged = pd.merge(merged, 
          tmp[["Location","LocationHierarchy","Frequency", col_to_export]], 
          on=["Location","LocationHierarchy","Frequency"], suffixes=("", get_date(files[i])), how="outer")
    
merged[col_to_export + get_date(files[2])] = merged[col_to_export]
del merged[col_to_export]


In [12]:
merged["venezuelans_hist_min"] = merged.min(axis=1)
merged["venezuelans_hist_max"] = merged.max(axis=1)
merged.to_csv("May26_Jun29.csv")

In [14]:
# Creates final merged file
infile1 = "May26_Jun29.csv"
infile2 = "merged_06_29.csv"

df1 = pd.read_csv(infile1)
df2 = pd.read_csv(infile2)

merged_all = pd.merge(df1, df2, how="outer", on=["Location","LocationHierarchy","Frequency"], suffixes=(get_date(files[0]), (get_date(files[1]))))
del merged_all["Unnamed: 0"]
merged_all.fillna(0.0).to_csv("merged_06_29+.csv")


In [15]:
merged_all.head()

,Location,LocationHierarchy,Frequency,venezuelans05_26,venezuelans05_27,venezuelans05_29,venezuelans05_30,venezuelans05_31,venezuelans06_01,venezuelans06_03,...,%diff_audience_adolecent,%diff_audience_young_adult,%diff_audience_adult,%diff_audience_middle_age,%diff_audience_elder,%diff_audience_graduated,%diff_audience_high_school,%diff_audience_iOS,%diff_audience_Android,%diff_audience_Other
0,"Cúcuta, Norte de Santander, Colombia",City,Monthly,55000.0,55000.0,54000.0,54000.0,54000.0,59000.0,58000.0,...,-2.51,5.56,4.11,-6.71,-0.44,2.25,-2.25,-2.42,-6.03,8.45
1,"Cúcuta, Norte de Santander, Colombia",City,Daily,29585.0,30388.0,28160.0,29075.0,29792.0,31982.0,31576.0,...,-1.84,4.63,4.07,-6.22,-0.63,2.96,-2.96,-2.43,-10.44,12.87
2,"Arauca, Arauca, Colombia",City,Monthly,6100.0,6100.0,5900.0,5900.0,5900.0,6500.0,6500.0,...,-1.42,1.65,-4.47,-5.92,10.16,-0.48,0.48,8.51,-12.51,4.00
3,"Arauca, Arauca, Colombia",City,Daily,2853.0,2963.0,2417.0,2532.0,2530.0,2560.0,2659.0,...,-6.72,2.65,-9.53,-2.80,16.41,-2.49,2.49,-4.08,-6.85,10.92
4,"Riohacha, La Guajira, Colombia",City,Monthly,13000.0,13000.0,13000.0,13000.0,13000.0,14000.0,14000.0,...,-4.75,4.34,1.46,-6.04,4.99,-3.24,3.24,2.72,-13.11,10.38


In [23]:
month = merged_all[merged_all["Frequency"] == "Monthly"]
month[["Location","%diff_audience_man"]].sort_values(by="%diff_audience_man")

,Location,%diff_audience_man
74,"Vichada, Colombia",-9.09
72,"Vaupes, Colombia",-8.82
76,"Casanare, Colombia",-4.69
90,Guyana,-2.25
52,"Meta, Colombia",-1.93
94,United States,-1.76
42,"Guaviare, Colombia",-1.43
44,"Guainia, Colombia",-0.93
22,"Arauca, Colombia",-0.61
8,Venezuela,-0.36
